# Challenge LATAM Data Engineer

In [1]:
import ujson as json
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Tuple
import emoji
import pandas as pd

## EDA

In [8]:

df = pd.read_json('tweets.json', lines=True)


In [13]:
pd.set_option('display.max_columns', None)


In [14]:
df.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,0,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,0,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24 09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,0,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,0,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24 09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,0,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None


In [16]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117407 entries, 0 to 117406
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   url              117407 non-null  object             
 1   date             117407 non-null  datetime64[ns, UTC]
 2   content          117407 non-null  object             
 3   renderedContent  117407 non-null  object             
 4   id               117407 non-null  int64              
 5   user             117407 non-null  object             
 6   outlinks         117407 non-null  object             
 7   tcooutlinks      117407 non-null  object             
 8   replyCount       117407 non-null  int64              
 9   retweetCount     117407 non-null  int64              
 10  likeCount        117407 non-null  int64              
 11  quoteCount       117407 non-null  int64              
 12  conversationId   117407 non-null  int64              
 13 

## Funciones Q1

### q1_time

- 

In [2]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_counts = defaultdict(int)
    user_counts_per_date = defaultdict(Counter)
    
    with open(file_path, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            date_str = tweet["date"]
            date = datetime.fromisoformat(date_str).date()
            user = tweet["user"]["username"]
            
            date_counts[date] += 1
            user_counts_per_date[date][user] += 1

    top_dates = sorted(date_counts, key=date_counts.get, reverse=True)[:10]
    
    result = []
    for date in top_dates:
        top_user = user_counts_per_date[date].most_common(1)[0][0]
        result.append((date, top_user))
        
    return result


In [3]:
q1_time('tweets.json')

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

### q1_memory

-

In [5]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_counts = defaultdict(int)
    
    with open(file_path, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            date_str = tweet["date"]
            date = datetime.fromisoformat(date_str).date()
            
            date_counts[date] += 1

    top_dates = sorted(date_counts, key=date_counts.get, reverse=True)[:10]
    
    result = []
    for date in top_dates:
        user_counts = Counter()
        
        with open(file_path, 'r') as f:
            for line in f:
                tweet = json.loads(line)
                curr_date_str = tweet["date"]
                curr_date = datetime.fromisoformat(curr_date_str).date()
                if curr_date == date:
                    user = tweet["user"]["username"]
                    user_counts[user] += 1

        top_user = user_counts.most_common(1)[0][0]
        result.append((date, top_user))
        
    return result


In [6]:
q1_memory('tweets.json')

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

## Funciones Q2

### q2_time

In [2]:
def q2_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    emoji_set = set(emoji.EMOJI_DATA.keys())
    emoji_counts = Counter()

    with open(file_path, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            content = tweet["content"]
            emojis_list = [c for c in content if c in emoji_set]
            emoji_counts.update(emojis_list)

    return emoji_counts.most_common(10)



In [3]:
q2_time('tweets.json')

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🏻', 2080),
 ('❤', 1779),
 ('🤣', 1668),
 ('🏽', 1218),
 ('👇', 1108)]

### q2_memory

In [6]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    emoji_counts = Counter()

    def read_jsonl(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                yield json.loads(line)

    for tweet in read_jsonl(file_path):
        content = tweet["content"]
        for c in content:
            if emoji.EMOJI_DATA.get(c):
                emoji_counts[c] += 1

    return emoji_counts.most_common(10)

In [7]:
q2_memory('tweets.json')

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🏻', 2080),
 ('❤', 1779),
 ('🤣', 1668),
 ('🏽', 1218),
 ('👇', 1108)]

## Funciones Q3

### q3_time

-

In [8]:


def q3_time(file_path: str) -> List[Tuple[str, int]]:
    mentions_counter = Counter()
    
    with open(file_path, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            content = tweet["content"]
            mentions = [word[1:] for word in content.split() if word.startswith('@')]
            mentions_counter.update(mentions)

    return mentions_counter.most_common(10)


In [10]:
q3_time('tweets.json')

[('narendramodi', 2201),
 ('Kisanektamorcha', 1822),
 ('RakeshTikaitBKU', 1599),
 ('PMOIndia', 1387),
 ('RahulGandhi', 1093),
 ('RaviSinghKA', 1002),
 ('GretaThunberg', 999),
 ('UNHumanRights', 956),
 ('rihanna', 944),
 ('meenaharris', 911)]

### q3_memory

-

In [11]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    mentions_counter = defaultdict(int)

    def read_jsonl(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                yield json.loads(line)

    for tweet in read_jsonl(file_path):
        content = tweet["content"]
        mentions = (word[1:] for word in content.split() if word.startswith('@'))
        for mention in mentions:
            mentions_counter[mention] += 1

    return sorted(mentions_counter.items(), key=lambda x: x[1], reverse=True)[:10]


In [12]:
q3_memory('tweets.json')

[('narendramodi', 2201),
 ('Kisanektamorcha', 1822),
 ('RakeshTikaitBKU', 1599),
 ('PMOIndia', 1387),
 ('RahulGandhi', 1093),
 ('RaviSinghKA', 1002),
 ('GretaThunberg', 999),
 ('UNHumanRights', 956),
 ('rihanna', 944),
 ('meenaharris', 911)]